Create the tables for this section.

In [1]:
%load_ext sql       
# Connect to an empty SQLite database
%sql sqlite://

'Connected: None@None'

In [2]:
%%sql
DROP TABLE IF EXISTS Purchase;
-- Create tables
CREATE TABLE Purchase (
    Product  VARCHAR(255), 
    Date     DATE, 
    Price    FLOAT,
    Quantity INT
);

-- Insert tuples
INSERT INTO Purchase VALUES ('Bagel', '2017-10-21', 1, 20);
INSERT INTO Purchase VALUES ('Bagel', '2017-10-25', 1.5, 20);
INSERT INTO Purchase VALUES ('Banana', '2017-10-03', 0.5, 10);
INSERT INTO Purchase VALUES ('Banana', '2017-10-10', 1, 10);

SELECT * FROM Purchase;

Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
Done.


Product,Date,Price,Quantity
Bagel,2017-10-21,1.0,20
Bagel,2017-10-25,1.5,20
Banana,2017-10-03,0.5,10
Banana,2017-10-10,1.0,10


## Aggregation Operations

SQL support several __aggregation__ operations
* SUM, COUNT, MIN, MAX, AVG
* Except COUNT, all aggregations apply to a single attribute

### COUNT

Syntax
```mysql
SELECT COUNT(column_name)
FROM   table_name
WHERE  condition;
``` 

> __Example:__ Find the number of purchases

| Product |    Date    | Price | Quantity |
|---------|------------|-------|----------|
| Bagel   | 2017-10-21 | 1     | 20       |
| Bagel   | 2017-10-25 | 1.5   | 20       |
| Banana  | 2017-10-03 | 0.5   | 10       |
| Banana  | 2017-10-10 | 1     | 10       |

In [3]:
%%sql
SELECT COUNT(Product)
FROM   Purchase;

Done.


COUNT(Product)
4


* Count applies to duplicates, unless otherwise stated
  * Same as ```COUNT(*)```. Why?

> __Example:__ Find the number of __different__ product purchases


| Product |    Date    | Price | Quantity |
|---------|:----------:|------:|:--------:|
| Bagel   | 2017-10-21 | 1.0   | 20       |
| Bagel   | 2017-10-25 | 1.5   | 20       |
| Banana  | 2017-10-03 | 0.5   | 10       |
| Banana  | 2017-10-10 | 1.0   | 10       |

* Use DISTINCT

In [4]:
%%sql
SELECT COUNT(DISTINCT Product)
FROM   Purchase;

Done.


COUNT(DISTINCT Product)
2


### SUM

Syntax
```mysql
SELECT SUM(column_name)
FROM   table_name
WHERE  condition;
```

> __Example:__ How many units of all products have been purchased?

| Product |    Date    | Price | Quantity |
|---------|:----------:|------:|:--------:|
| Bagel   | 2017-10-21 | 1.0   | 20       |
| Bagel   | 2017-10-25 | 1.5   | 20       |
| Banana  | 2017-10-03 | 0.5   | 10       |
| Banana  | 2017-10-10 | 1.0   | 10       |

In [5]:
%%sql
SELECT SUM(Quantity)
FROM   Purchase;

Done.


SUM(Quantity)
60


> __Example:__ How many Bagels have been purchased?

| Product |    Date    | Price | Quantity |
|---------|:----------:|------:|:--------:|
| Bagel   | 2017-10-21 | 1.0   | 20       |
| Bagel   | 2017-10-25 | 1.5   | 20       |
| Banana  | 2017-10-03 | 0.5   | 10       |
| Banana  | 2017-10-10 | 1.0   | 10       |

In [6]:
%%sql
SELECT SUM(Quantity)
FROM   Purchase
WHERE  Product = 'Bagel';

Done.


SUM(Quantity)
40


## AVG

Syntax
```mysql
SELECT AVG(column_name)
FROM   table_name
WHERE  condition;
```

> __Example:__ What is the average sell price of Bagels?

| Product |    Date    | Price | Quantity |
|---------|:----------:|------:|:--------:|
| Bagel   | 2017-10-21 | 1.0   | 20       |
| Bagel   | 2017-10-25 | 1.5   | 20       |
| Banana  | 2017-10-03 | 0.5   | 10       |
| Banana  | 2017-10-10 | 1.0   | 10       |

In [7]:
%%sql
SELECT AVG(Price)
FROM   Purchase
WHERE  Product = 'Bagel';

Done.


AVG(Price)
1.25


### Simple Aggregations

> __Example:__ Total earnings from Bagels sold?

| Product |    Date    | Price | Quantity |
|---------|:----------:|------:|:--------:|
| Bagel   | 2017-10-21 | 1.0   | 20       |
| Bagel   | 2017-10-25 | 1.5   | 20       |
| Banana  | 2017-10-03 | 0.5   | 10       |
| Banana  | 2017-10-10 | 1.0   | 10       |

In [8]:
%%sql
SELECT SUM(Price * Quantity)
FROM   Purchase
WHERE  Product = 'Banana';

Done.


SUM(Price * Quantity)
15.0


## GROUP BY

Used with aggregate functions (COUNT, MAX, MIN, SUM, AVG) to group the result-set by one or more columns.

Syntax
```mysql
SELECT   column_name(s)
FROM     table_name
WHERE    condition
GROUP BY column_name(s)
[ORDER BY column_name(s)];
```

> __Example:__ Find total sales after October 1st. per product

| Product |    Date    | Price | Quantity |
|---------|:----------:|------:|:--------:|
| Bagel   | 2017-10-21 | 1.0   | 20       |
| Bagel   | 2017-10-25 | 1.5   | 20       |
| Banana  | 2017-10-03 | 0.5   | 10       |
| Banana  | 2017-10-10 | 1.0   | 10       |

In [9]:
%%sql
SELECT   Product, SUM(price * quantity) AS TotalSales
FROM     Purchase
WHERE    Date > '2017-10-01'
GROUP BY Product;

Done.


Product,TotalSales
Bagel,50.0
Banana,15.0


### Grouping and Aggregation: Semantics of the Query

__1.__ Compute the FROM and WHERE clauses

In [10]:
%%sql
SELECT *
FROM   Purchase
WHERE  Date > '2017-10-01'

Done.


Product,Date,Price,Quantity
Bagel,2017-10-21,1.0,20
Bagel,2017-10-25,1.5,20
Banana,2017-10-03,0.5,10
Banana,2017-10-10,1.0,10


__2.__ Group attributes according to GROUP BY

|__Product__| __Date__ |__Price__|__Quantity__|
|:-------:|:----------:|:-----:|:--------:|
|  Bagel  | 2017-10-21 |   1   |    20    |
|         | 2017-10-25 |  1.5  |    20    |
|  Banana | 2017-10-03 |  0.5  |    10    |
|         | 2017-10-10 |   1   |    10    |


__Caution:__ SQL _only_ displays one row if no aggregation function is used

In [11]:
%%sql
SELECT   *
FROM     Purchase
WHERE    Date > '2017-10-01'
GROUP BY Product;

Done.


Product,Date,Price,Quantity
Bagel,2017-10-25,1.5,20
Banana,2017-10-10,1.0,10


In [12]:
%%sql
SELECT   Product, Count(Product)
FROM     Purchase
WHERE    Date > '2017-10-01'
GROUP BY Product;

Done.


Product,Count(Product)
Bagel,2
Banana,2


__3.__ Compute the SELECT clause: grouped attributes and aggregates

| Product |    Date    | Price | Quantity |
|---------|:----------:|------:|:--------:|
| Bagel   | 2017-10-21 | 1.0   | 20       |
| Bagel   | 2017-10-25 | 1.5   | 20       |
| Banana  | 2017-10-03 | 0.5   | 10       |
| Banana  | 2017-10-10 | 1.0   | 10       |

In [13]:
%%sql  -- Find total sales after '2017-10-01' per product

SELECT   Product, SUM(price * quantity) AS TotalSales
FROM     Purchase
WHERE    Date > '2017-10-01'
GROUP BY Product;

Done.


Product,TotalSales
Bagel,50.0
Banana,15.0


### GROUP BY vs Nested Queries

```mysql
SELECT   Product, SUM(price * quantity) AS TotalSales
FROM     Purchase
WHERE    Date > '10/1'
GROUP BY Product;
```

In [14]:
%%sql
SELECT DISTINCT x.Product, (SELECT Sum(y.price*y.quantity)
                                   FROM Purchase y
                                   WHERE x.product = y.product
                                         AND y.date > '2017-10-01') AS TotalSales
FROM  Purchase x
WHERE x.date > '2017-10-01';

Done.


Product,TotalSales
Bagel,50.0
Banana,15.0


## HAVING

* HAVING clauses contain conditions on __aggregates__
* WHERE clauses condition on __individual tuples__

Syntax
```mysql
SELECT   column_name(s)
FROM     table_name
WHERE    condition
GROUP BY column_name(s)
HAVING   condition
[ORDER BY column_name(s)];
```

> __Example:__ Same query as before, except that we consider only products with more than 30 units sold

| Product |    Date    | Price | Quantity |
|---------|:----------:|------:|:--------:|
| Bagel   | 2017-10-21 | 1.0   | 20       |
| Bagel   | 2017-10-25 | 1.5   | 20       |
| Banana  | 2017-10-03 | 0.5   | 10       |
| Banana  | 2017-10-10 | 1.0   | 10       |

In [15]:
%%sql
SELECT   Product, SUM(price * quantity) AS TotalSales
FROM     Purchase
WHERE    Date > '2017-10-01'
GROUP BY Product
HAVING SUM(Quantity) > 30;

Done.


Product,TotalSales
Bagel,50.0


### Exercise II

An organism that sells tickets for football matches uses a database with the following relational schema:

```
Match(Match_ID, Date, Hour, Stadium_ID, Team_ID)
Team(Team_ID, Name, City)
Stadium(Stadium_ID, Name, Address, Capacity, Team_ID)
Ticket(Ticket_ID, Match_ID, Place_Number, Category, Price)
Sell(Sell_ID, Sell_Date, Ticket_ID, Payment_Method)
```

Write the following query in SQL:
> What are the name(s) of the stadium(s) with **the** largest capacity?

___

# Advanced\* Topics

In this section
* Relational Division is SQL
* Nulls (revisited)
* Outer Joins

## Relational Division in SQL

* Not supported as a primitive operator, but useful for expressing queries like:

> _"Find suppliers who sell the x parts..."_

> _"Find buyers who bought all products from a given category..."_

* Let $A$ have 2 fields, $x$ and $y$, $B$ have only field $y$
```mysql
A(x, y)
B(y)
```
  * $A/B$ contains all $x$ tuples such that for every $y$ tuple in $B$, there is an $xy$ tuple in $A$
  * Or: If the set of $y$ values associated with an $x$ value in $A$ contains all $y$ values in $B$, the $x$ value is in $A/B$.

__Classic Option 1__
```mysql
SELECT T1.x
FROM A AS T1
WHERE NOT EXISTS( SELECT T2.y
                  FROM B AS T2
                  EXCEPT
                  SELECT T3.y
                  FROM A AS T3
                  WHERE T3.y=T1.y);
```

__Classic Option 2__ (without EXCEPT)
```mysql
SELECT DISTINCT T1.x
FROM A AS T1
WHERE NOT EXISTS(SELECT T2.y
                 FROM B AS T2
                 WHERE NOT EXISTS (SELECT T3.x
                                   FROM A AS T3
                                   WHERE T3.x=T1.x
                                   AND T3.y=T2.y
                                  )
                );
```

> __Example:__ Find Establishments which sell all products
```
Establishment(eid, ename)
Sells(eid, pname)
Products(pname)
```

Classic Option 2 (without EXCEPT)
```mysql
SELECT DISTINCT E.ename
FROM Establishment AS E
WHERE NOT EXISTS (SELECT p.pname
                  FROM Products3 AS P 
                  WHERE NOT EXISTS (SELECT S.eid
                                    FROM Sells AS S
                                    WHERE S.pname=P.pname
                                    AND S.eid=E.eid
                                   )
                );
```

Classic Option 2 (without EXCEPT)
```mysql
SELECT DISTINCT E.ename
FROM Establishment AS E
WHERE NOT EXISTS (SELECT p.pname
                  FROM Products3 AS P 
                  WHERE NOT EXISTS (SELECT S.eid
                                    FROM Sells AS S
                                    WHERE S.pname=P.pname
                                    AND S.eid=E.eid
                                   )
                );
```

* Semantics:
  * _Establishment_ E such that...
    
    ... there is no _Product_ P...
 
    ...... without a _Sells_ tuple showing that E sells P 

> __Example:__ Find Establishments which sell all products
![division_example](figures/sql_division_example.png)

In [16]:
%%sql
DROP TABLE IF EXISTS Establishment;
-- Create tables
CREATE TABLE Establishment (
    eid INT,
    ename VARCHAR);

DROP TABLE IF EXISTS Sells;
-- Create tables
CREATE TABLE Sells (
    eid INT,
    pname VARCHAR);

DROP TABLE IF EXISTS Products1;
-- Create tables
CREATE TABLE Products1 (
    pname VARCHAR);

DROP TABLE IF EXISTS Products2;
-- Create tables
CREATE TABLE Products2 (
    pname VARCHAR);

DROP TABLE IF EXISTS Products3;
-- Create tables
CREATE TABLE Products3 (
    pname VARCHAR);

-- Insert tuples
INSERT INTO Establishment VALUES (1, 'Carrefour');
INSERT INTO Establishment VALUES (2, 'Franprix');
INSERT INTO Establishment VALUES (3, 'Boulangerie');
INSERT INTO Establishment VALUES (4, 'Biocoop');
INSERT INTO Sells VALUES (1, 'Wine');
INSERT INTO Sells VALUES (1, 'Bread');
INSERT INTO Sells VALUES (1, 'Cheese');
INSERT INTO Sells VALUES (1, 'Meat');
INSERT INTO Sells VALUES (2, 'Wine');
INSERT INTO Sells VALUES (2, 'Bread');
INSERT INTO Sells VALUES (3, 'Bread');
INSERT INTO Sells VALUES (4, 'Bread');
INSERT INTO Sells VALUES (4, 'Meat');
INSERT INTO Products1 VALUES ('Bread');
INSERT INTO Products2 VALUES ('Bread');
INSERT INTO Products2 VALUES ('Meat');
INSERT INTO Products3 VALUES ('Wine');
INSERT INTO Products3 VALUES ('Bread');
INSERT INTO Products3 VALUES ('Meat');

Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [17]:
%%sql
SELECT * FROM Establishment;

Done.


eid,ename
1,Carrefour
2,Franprix
3,Boulangerie
4,Biocoop


In [18]:
%%sql
SELECT * FROM Sells;

Done.


eid,pname
1,Wine
1,Bread
1,Cheese
1,Meat
2,Wine
2,Bread
3,Bread
4,Bread
4,Meat


In [19]:
%%sql
SELECT * FROM Products3

Done.


pname
Wine
Bread
Meat


> __Example:__ Find Establishments which sell all products
![division_example](figures/sql_division_example.png)

In [20]:
%%sql  -- Change bellow to query Products[1,2,3]
SELECT DISTINCT E.ename
FROM Establishment AS E
WHERE NOT EXISTS (SELECT P.pname
                  FROM Products3 AS P 
                  WHERE NOT EXISTS (SELECT S.eid
                                    FROM Sells AS S
                                    WHERE S.pname=P.pname
                                    AND S.eid=E.eid
                                   )
                );

Done.


ename
Carrefour


> __Exercise:__ Write the same query with EXCEPT (Classic Option 1)
```
Establishment(eid, ename)
Sells(eid, pname)
Products(pname)
```
___
```mysql
SELECT T1.x
FROM A AS T1
WHERE NOT EXISTS( SELECT T2.y
                  FROM B AS T2
                  EXCEPT
                  SELECT T3.y
                  FROM A AS T3
                  WHERE T3.y=T1.y);
```

> __Example:__ Find Establishments which sell all products
![division_example](figures/sql_division_example.png)

In [21]:
%%sql
-- Write the same query with EXCEPT (Classic Option 1)


Done.


[]

### Exercise III

An organism that sells tickets for football matches uses a database with the following relational schema:

```
Match(Match_ID, Date, Hour, Stadium_ID, Team_ID)
Team(Team_ID, Name, City)
Stadium(Stadium_ID, Name, Address, Capacity, Team_ID)
Ticket(Ticket_ID, Match_ID, Place_Number, Category, Price)
Sell(Sell_ID, Sell_Date, Ticket_ID, Payment_Method)
```

Write the following query in SQL:
> What are the teams that will play at least once in all the stadiums?

### Exercise IV

An organism that sells tickets for football matches uses a database with the following relational schema:

```
Match(Match_ID, Date, Hour, Stadium_ID, Team_ID)
Team(Team_ID, Name, City)
Stadium(Stadium_ID, Name, Address, Capacity, Team_ID)
Ticket(Ticket_ID, Match_ID, Place_Number, Category, Price)
Sell(Sell_ID, Sell_Date, Ticket_ID, Payment_Method)
```

Write the following query in SQL:
> What are the dates and identifiers of matches for which there are no more tickets to sell?

### Yet another option

["A Simpler (and Better) SQL Approach to Relational Division"](https://users.dcc.uchile.cl/~cgutierr/cursos/BD/divisionSQL.pdf)

Journal of Information Systems Education, Vol. 13(2)

## Null Values

* For _numerical operations_, NULL -> NULL:
  * If x is NULL then ```4*(3-x)/7``` is still NULL

* For _boolean operations_, in SQL there are three values:

```
FALSE  = 0
UNKNOWN = 0.5
TRUE    = 1
```

* If x is NULL then ```x = 'Joe'``` is UNKNOWN

```
C1 AND C2   = min(C1, C2)
C1 OR C2    = max(C1, C2)
NOT C1      = 1 – C1
```

>  __Example:__
```mysql
SELECT *
FROM   Person
WHERE  (age < 25)
  AND  (height > 6 AND weight > 190);
```
Won't return: 
- age=20
- height=NULL  <--
- weight=200

__Rule in SQL:__ include only tuples that yield TRUE (1.0)

> __Example:__ Unexpected behavior
```mysql
SELECT *
FROM   Person
WHERE  age < 25 OR age >= 25;
```
Some tuples from _Person_ are not included

Test for NULL explicitly:
* x IS NULL
* x IS NOT NULL


>```mysql
SELECT *
FROM   Person
WHERE  age < 25 OR age >= 25 OR age IS NULL;
```
Now it includes all tuples in _Person_

## Inner Joins + NULLS = Lost data?

* By default, joins in SQL are __inner joins__

> __Example:__ Find Products (Name) and the Stores where they are sold.
```
Product(name, category)
Purchase(prodName, store)
```

> __Example:__ Find Products (Name) and the Stores where they are sold.
```
Product(name, category)
Purchase(prodName, store)
```
Syntax 1
```mysql
SELECT Product.name, Purchase.store
FROM   Product
JOIN   Purchase ON Product.name = Purchase.prodName;
```
Syntax 2
```mysql
SELECT Product.name, Purchase.store
FROM   Product, Purchase
WHERE  Product.name = Purchase.prodName;
```
* Both equivalent, both _inner joins_

* __However:__ Products that never sold (with no Purchase tuple) will be lost!

## Outer Joins
* An __outer join__ returns tuples from the joined relations that don’t have a corresponding tuple in the other relations
  * i.e. If we join relations A and B on A.x = B.x, and there is an entry in A with x=5, but none in B with x=5
  LEFT [OUTER] JOIN will return a tuple __(5, NULL)__
 
Syntax
```mysql
SELECT column_name(s)
FROM   table1
LEFT OUTER JOIN table2 ON table1.column_name = table2.column_name;
```

In [22]:
%%sql

-- Create tables
DROP TABLE IF EXISTS Product;
CREATE TABLE Product (
    name VARCHAR(255) PRIMARY KEY,
    category VARCHAR(255)
);

DROP TABLE IF EXISTS Purchase;
CREATE TABLE Purchase(
    prodName varchar(255),
    store varchar(255)
);

-- Insert tuples
INSERT INTO Product VALUES ('Gizmo', 'Gadget');
INSERT INTO Product VALUES ('Camera', 'Photo');
INSERT INTO Product VALUES ('OneClick', 'Photo');

INSERT INTO Purchase VALUES ('Gizmo', 'Wiz');
INSERT INTO Purchase VALUES ('Camera', 'Ritz');
INSERT INTO Purchase VALUES ('Camera', 'Wiz');

Done.
Done.
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [23]:
%%sql
SELECT *
FROM   Product;

Done.


name,category
Gizmo,Gadget
Camera,Photo
OneClick,Photo


In [24]:
%%sql
SELECT *
FROM   Purchase;

Done.


prodName,store
Gizmo,Wiz
Camera,Ritz
Camera,Wiz


In [25]:
%%sql
SELECT Product.name, Purchase.store
FROM   Product
LEFT OUTER JOIN Purchase
ON Product.name = Purchase.prodName;

Done.


name,store
Camera,Ritz
Camera,Wiz
Gizmo,Wiz
OneClick,None


## Outer Joins
* __Left outer join__
  * Include the left tuple even if there is no match
* __Right outer join__
  * Include the right tuple even if there is no match
* __Full outer join__
  * Include both left and right tuples even if there is no match

___

# Summary

* The relational model has rigorously defined query languages that are simple and powerful.
* Several ways of expressing a given query
  * A query optimizer should choose the most efficient version.
* SQL is the lingua franca (common language) for accessing relational database systems.
* SQL is a rich language that handles the way data is processed ___declaratively___
  * Expresses the logic of a computation without describing its control flow

___

In [26]:
# Modify the css style
# from IPython.core.display import HTML
# def css_styling():
#     styles = open("./style/custom.css").read()
#     return HTML(styles)
# css_styling()